<a href="https://colab.research.google.com/github/simulate111/Deep-Learning-in-Human-Language-Technology/blob/main/exercise_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

In [2]:
from transformers import pipeline

p = pipeline(model="EleutherAI/gpt-neo-2.7B")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Let's try simple QA first.

In [3]:
qa_prompt_zero = """
Answer the following question.

Question: What is the currency of France?
Answer: """

qa_prompt_one = """
Answer the following question.

Question: What is the currency of US?
Answer: Dollar

Question: What is the currency of France?
Answer: """


print("Generated zero:", p(qa_prompt_zero, do_sample=False, max_new_tokens=50)[0]["generated_text"])
print("---------")
print("Generated one:", p(qa_prompt_one, do_sample=False, max_new_tokens=50)[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated zero: 
Answer the following question.

Question: What is the currency of France?
Answer:                                                   
---------


KeyboardInterrupt: 

In [4]:
qa_prompt2_zero = """
Answer the following question.

Question: Who is the President of Finland?
Answer: """

qa_prompt2_one = """
Answer the following question.

Question: Who is the President of US?
Answer: Joe Biden

Question: Who is the President of Finland?
Answer: """

print("Generated zero:", p(qa_prompt2_zero, do_sample=False, max_new_tokens=50)[0]["generated_text"])
print("---------")
print("Generated one:", p(qa_prompt2_one, do_sample=False, max_new_tokens=50)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [5]:
# let's try something else
qa_prompt2_zero = "The president of Finland is"

qa_prompt2_one = "The president of US is Joe Biden. The president of Finland is"

print("Generated zero:", p(qa_prompt2_zero, do_sample=False, max_new_tokens=50)[0]["generated_text"])
print("---------")
print("Generated one:", p(qa_prompt2_one, do_sample=False, max_new_tokens=50)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

# Let's try classification next
  * Language identification

In [ ]:
lang_prompt_zero = """
Identify the language of the given text.

Text: L'intelligence artificielle est un ordinateur ou un programme informatique capable d'exécuter des fonctions qui peuvent être considérées comme intelligentes.
Language:"""

lang_prompt_few = """
Identify the language of the given text.

Text: Artificial intelligence is a computer or computer program capable of performing functions that can be considered intelligent.
Language: English

Text: La inteligencia artificial es un ordenador o programa informático capaz de realizar funciones que pueden considerarse inteligentes.
Language: Spanish

Text: L'intelligence artificielle est un ordinateur ou un programme informatique capable d'exécuter des fonctions qui peuvent être considérées comme intelligentes.
Language: """

print("Generated zero:", p(lang_prompt_zero, do_sample=False, max_new_tokens=50)[0]["generated_text"])
print("---------")
print("Generated one:", p(lang_prompt_few, do_sample=False, max_new_tokens=50)[0]["generated_text"])

# Artificial intelligence is a computer or computer program capable of performing functions that can be considered intelligent.
# L'intelligence artificielle est un ordinateur ou un programme informatique capable d'exécuter des fonctions qui peuvent être considérées comme intelligentes.
# La inteligencia artificial es un ordenador o programa informático capaz de realizar funciones que pueden considerarse inteligentes.

In [ ]:
lang_prompt_zero = "L'intelligence artificielle est un ordinateur ou un programme informatique capable d'exécuter des fonctions qui peuvent être considérées comme intelligentes. This sentence is written in"

print("Generated zero:", p(lang_prompt_zero, do_sample=False, max_new_tokens=50)[0]["generated_text"])

## Notes
* I did not experiment with the generation parameters, but there are some important parameters controlling the generation:
* `do_sample=False` means greedy decoding, i.e. always choose a token with the highest probability as the next token
* `multinomial sampling`: randomly selects the next token based on the probability distribution over the entire vocabulary given by the model
  - `top_k`: sample only from K most likely next words, and redistribute the probability mass among those words only
  - `top_p`: sample from the smallest possible set of words whose cumulative probability exceeds the probability p, and redistribute the probability mass among this set of words
  - `temperature`: increase or decrease the confidence of a model by making the probability distribution sharper (low temperature values) or smoother (high temperature values), incorporated into the softmax function
* More about this: [https://huggingface.co/blog/how-to-generate](https://huggingface.co/blog/how-to-generate)
